In [ ]:
pip install -r requirements.txt

## Document Text processing and embedding creation

Steps : 
1. Open a pdf
2. Format the text for the embedding model (split into chunk of text)
3. Embed text chunks with embedding model
4. Save embeddings to a file for later use

Requirements : 
1. pdf 
2. embedding model of choice

In [3]:
import os
import requests

#the pdf document path
pdf_doc = 'C:/Users/yashb/Downloads/RAG testing/pdfs/Book-2Designing-data-intensive-applications.pdf'

if not os.path.exists(pdf_doc):
    print("[INFO] file does not exist, Downloading using the url....")
    url = "Put the download url of the pdf"
    filename = pdf_doc

    #get request for pdf download
    response = requests.get(url)
    #check if the get request was successful
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f" [INFO] The file have been downloaded and saved as {filename}")
    else:
        print(f"Failed to download the file. Request ended with status code {respose.status_code}")
else:
    print(f"File {pdf_doc} exists")

File C:/Users/yashb/Downloads/RAG testing/pdfs/Book-2Designing-data-intensive-applications.pdf exists


In [4]:
import pymupdf
from tqdm.auto import tqdm
import re
   
def clean_text(text: str) -> str:
    # Remove hyphenation at line breaks
    text = re.sub(r'-\n', '', text)
    
    # Merge lines (replace single newlines with space)
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove footers like "216\n|\nChapter 6: Partitioning or opposite"
    text = re.sub(r'\d+\s*\n\|\n.*?$', '', text.strip(), flags=re.MULTILINE)
    #text = re.sub(r"\s*\|\s*\d+\s*$", "", text)
    text = re.sub(r"^\s*\d+\s*\n\s*\|\s*\n\s*Chapter\s+\d+:.*(?:\n)?", '', text.strip(), flags=re.MULTILINE)

    # Final strip
    return text.strip()

#or till doc.page_count currently removing the unimportant pages from this book
def open_read_clean_pdf(pdf_doc: str) -> list[dict]:
    resource = pymupdf.open(pdf_doc)
    all_text = []
    for page_num, page in tqdm(enumerate(resource)):
        text = page.get_text()
        text = clean_text(text=text)
        all_text.append(
            {
                'page_no': page_num - 21, #There is unuseful stuff till page 23 so you can do whatever with this, here the page number starts at page 22 or 23
                'page_char_cnt': len(text),
                'page_word_cnt': len(text.split(" ")),
                'page_sentence_cnt': len(text.split(". ")),
                'page_token_cnt': len(text)/4,   #English - token is approx 4 characters
                'text': text                
            }
        )
    return all_text

all_text = open_read_clean_pdf(pdf_doc = pdf_doc)


C:\Users\yashb\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
613it [00:01, 452.20it/s]


In [5]:
all_text[25]
#len(all_text)

#resource = pymupdf.open(pdf_doc)
#for page in range(27,28):
#    page = doc.load_page(page)
#    text = page.get_text()
#    print(text)
  

{'page_no': 4,
 'page_char_cnt': 2919,
 'page_word_cnt': 464,
 'page_sentence_cnt': 22,
 'page_token_cnt': 729.75,
 'text': 'But reality is not that simple. There are many database systems with different charac‐ teristics, because different applications have different requirements. There are vari‐ ous approaches to caching, several ways of building search indexes, and so on. When building an application, we still need to figure out which tools and which approaches are the most appropriate for the task at hand. And it can be hard to combine tools when you need to do something that a single tool cannot do alone. This book is a journey through both the principles and the practicalities of data sys‐ tems, and how you can use them to build data-intensive applications. We will explore what different tools have in common, what distinguishes them, and how they achieve their characteristics. In this chapter, we will start by exploring the fundamentals of what we are trying to achieve: reliable,

In [6]:
import pandas as pd

df = pd.DataFrame(all_text)
df.describe()

,page_no,page_char_cnt,page_word_cnt,page_sentence_cnt,page_token_cnt
count,613.000000,613.000000,613.000000,613.000000,613.000000
mean,285.000000,2303.140294,371.422512,17.084829,575.785073
std,177.102136,703.855066,118.008208,14.287294,175.963767
min,-21.000000,0.000000,1.000000,1.000000,0.000000
25%,132.000000,2025.000000,307.000000,14.000000,506.250000
50%,285.000000,2560.000000,408.000000,17.000000,640.000000
75%,438.000000,2798.000000,457.000000,20.000000,699.500000
max,591.000000,3245.000000,550.000000,190.000000,811.250000


#Splitting pages into sentences (2 ways)
1. W splitting ". "
2. NLP library

In [7]:
from spacy.lang.en import English

nlp = English()

#Add a sentencizer pipeline
nlp.add_pipe("sentencizer")

for each_item in tqdm(all_text):
    each_item["sentences"] = list(nlp(each_item["text"]).sents)
    #spaCy dtype to string for sentences
    each_item["sentences"] = [str(sentence) for sentence in each_item["sentences"]]
    each_item["sentence_cnt_spacy"] = len(each_item["sentences"])

100%|██████████| 613/613 [00:02<00:00, 304.61it/s]


In [8]:
all_text[25]

{'page_no': 4,
 'page_char_cnt': 2919,
 'page_word_cnt': 464,
 'page_sentence_cnt': 22,
 'page_token_cnt': 729.75,
 'text': 'But reality is not that simple. There are many database systems with different charac‐ teristics, because different applications have different requirements. There are vari‐ ous approaches to caching, several ways of building search indexes, and so on. When building an application, we still need to figure out which tools and which approaches are the most appropriate for the task at hand. And it can be hard to combine tools when you need to do something that a single tool cannot do alone. This book is a journey through both the principles and the practicalities of data sys‐ tems, and how you can use them to build data-intensive applications. We will explore what different tools have in common, what distinguishes them, and how they achieve their characteristics. In this chapter, we will start by exploring the fundamentals of what we are trying to achieve: reliable,

In [9]:
df2 = pd.DataFrame(all_text)
df2.describe()

,page_no,page_char_cnt,page_word_cnt,page_sentence_cnt,page_token_cnt,sentence_cnt_spacy
count,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000
mean,285.000000,2303.140294,371.422512,17.084829,575.785073,16.045677
std,177.102136,703.855066,118.008208,14.287294,175.963767,6.361999
min,-21.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,132.000000,2025.000000,307.000000,14.000000,506.250000,14.000000
50%,285.000000,2560.000000,408.000000,17.000000,640.000000,18.000000
75%,438.000000,2798.000000,457.000000,20.000000,699.500000,20.000000
max,591.000000,3245.000000,550.000000,190.000000,811.250000,34.000000


In [10]:
#split size to turn group of sentences into chunks
split_size = 10 #research going on but you can take whatever

#Function to split the sentences recursively.
#i.e. 13 sentence = 10,10,3

def split_chunks(input_list: list[str], size: int) -> list[list[str]]:
    return [input_list[string:string+size] for string in range(0, len(input_list), size)]


for each_item in tqdm(all_text):
    each_item["sentence_chunks"] = split_chunks(input_list = each_item["sentences"], size = 10)
    each_item["chunk_size"] = len(each_item["sentence_chunks"])
    

100%|██████████| 613/613 [00:00<00:00, 301490.19it/s]


In [11]:
all_text[25]

{'page_no': 4,
 'page_char_cnt': 2919,
 'page_word_cnt': 464,
 'page_sentence_cnt': 22,
 'page_token_cnt': 729.75,
 'text': 'But reality is not that simple. There are many database systems with different charac‐ teristics, because different applications have different requirements. There are vari‐ ous approaches to caching, several ways of building search indexes, and so on. When building an application, we still need to figure out which tools and which approaches are the most appropriate for the task at hand. And it can be hard to combine tools when you need to do something that a single tool cannot do alone. This book is a journey through both the principles and the practicalities of data sys‐ tems, and how you can use them to build data-intensive applications. We will explore what different tools have in common, what distinguishes them, and how they achieve their characteristics. In this chapter, we will start by exploring the fundamentals of what we are trying to achieve: reliable,

In [12]:
#Making data more granular by splitting chunks into their own seperate paragraphs.
chunk_final_collection = []
for item in all_text:
    for chunk in tqdm(item['sentence_chunks']):
        chunk_dictionary = {}
        chunk_dictionary['page_no'] = item['page_no']
        chunk_paragraph = " ".join(chunk)
        fixed_text = re.sub(r'(\w+)[\-‐]\s+(\w+)', r'\1\2', chunk_paragraph)
        chunk_dictionary['chunk_paragraph'] = fixed_text
        chunk_dictionary['chunk_char_cnt'] = len(chunk_dictionary['chunk_paragraph'])
        chunk_dictionary['chunk_token_cnt'] = len(chunk_dictionary['chunk_paragraph']) / 4
        chunk_dictionary['chunk_word_cnt'] = len(chunk_dictionary['chunk_paragraph'].split(' '))
        chunk_final_collection.append(chunk_dictionary)

100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 1993.96it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 1996.81it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 2000.14it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 2001.58it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 2008.29it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 999

In [13]:
chunk_final_collection[56]

{'page_no': 14,
 'chunk_paragraph': 'Latency and response time Latency and response time are often used synonymously, but they are not the same. The response time is what the client sees: besides the actual time to process the request (the service time), it includes network delays and queueing delays. Latency is the duration that a request is waiting to be handled—during which it is latent, awaiting service [17]. Even if you only make the same request over and over again, you’ll get a slightly different response time on every try. In practice, in a system handling a variety of requests, the response time can vary a lot. We therefore need to think of response time not as a single number, but as a distribution of values that you can measure. In Figure 1-4, each gray bar represents a request to a service, and its height shows how long that request took. Most requests are reasonably fast, but there are occasional outliers that take much longer. Perhaps the slow requests are intrinsically m

In [14]:
df3 = pd.DataFrame(chunk_final_collection)
df3.describe()

,page_no,chunk_char_cnt,chunk_token_cnt,chunk_word_cnt
count,1239.000000,1239.000000,1239.000000,1239.000000
mean,286.660210,1135.793382,283.948345,182.500404
std,170.701532,534.753937,133.688484,83.523940
min,-21.000000,3.000000,0.750000,1.000000
25%,139.500000,869.500000,217.375000,135.500000
50%,288.000000,1206.000000,301.500000,196.000000
75%,433.000000,1443.000000,360.750000,238.000000
max,591.000000,2900.000000,725.000000,395.000000


SO the max chunk token count is 725. and the model for embedding we have selected all-mpnet-base-v2, you can check the stats - https://sbert.net/docs/sentence_transformer/pretrained_models.html
So vast majority of chunks are ~under 360 tokens which is fine as model's max sequence len is 384, but we will loose some tokens for 725 - 384 = 341 token loss.

can be resolved by replacing 10 sentences per chunk to less than that.
you can also experiment with overlaping of the sentences in chunks to get proper data together.
Also some chunks might be useless as you can see from the min token count so we can sample it and experiment with it to filter it.



In [15]:
filtered = df3[df3['chunk_token_cnt'] < 30]
for idx, row in filtered.iterrows():
    print(f"Page {row['page_no']} | {row['chunk_paragraph']}\n")

Page -6 | xiv | Preface

Page -5 | Preface | xv

Page -1 | You’re the best. Preface | xix

Page 62 | 62 | Chapter 2: Data Models and Query Languages

Page 64 | 64 | Chapter 2: Data Models and Query Languages

Page 66 | 66 | Chapter 2: Data Models and Query Languages

Page 93 | Transaction Processing or Analytics? | 93

Page 103 | Summary | 103

Page 114 | 114 | Chapter 4: Encoding and Evolution

Page 118 | Figure 4-2. Example record encoded using Thrift’s BinaryProtocol. 118 | Chapter 4: Encoding and Evolution

Page 141 | Morgan Kaufmann, 1999. ISBN: 978-0-122-33435-1 Summary | 141

Page 143 | 54] Fred Hebert: “Postscript: Maps,” learnyousomeerlang.com, April 9, 2014. Summary | 143

Page 151 | 151

Page 162 | 162 | Chapter 5: Replication

Page 174 | Even if the application checks availability before 174 | Chapter 5: Replication

Page 179 | Leaderless Replication | 179

Page 181 | Leaderless Replication | 181

Page 195 | sdf.org, January 4, 2016. Summary | 195

Page 196 | 196 | Chapter 

In [16]:
filtered = df3[df3['chunk_token_cnt'] > 30]
chunk_filtered_collection = filtered.to_dict(orient='records')
chunk_filtered_collection[:3]

[{'page_no': -19,
  'chunk_paragraph': 'Martin Kleppmann Designing Data-Intensive Applications The Big Ideas Behind Reliable, Scalable, and Maintainable Systems Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing',
  'chunk_char_cnt': 200,
  'chunk_token_cnt': 50.0,
  'chunk_word_cnt': 24},
 {'page_no': -18,
  'chunk_paragraph': '978-1-449-37332-0 [LSI] Designing Data-Intensive Applications by Martin Kleppmann Copyright © 2017 Martin Kleppmann. All rights reserved. Printed in the United States of America. Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472. O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are also available for most titles (http://oreilly.com/safari). For more information, contact our corporate/institutional sales department: 800-998-9938 or corporate@oreilly.com. Editors: Ann Spencer and Marie Beaugureau Indexer: Ellen Troutman-Zaig Production Editor: Kr

In [17]:
from sentence_transformers import SentenceTransformer
import time
embedding_model = SentenceTransformer(model_name_or_path = 'all-mpnet-base-v2')



In [18]:
%%time
#Doing one by one (can use cpu if you dont want to use gpu )
embedding_model.to('cuda')
for item in tqdm(chunk_filtered_collection):
    item['embedding'] = embedding_model.encode(item['chunk_paragraph'])

100%|██████████| 1180/1180 [01:59<00:00,  9.88it/s]

CPU times: total: 14min 7s
Wall time: 1min 59s


In [ ]:
pip install torch==2.3.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
pip install numpy==1.26.4

In [19]:
#Trying to do this in batches to check the performance

chunk_paragraph_batch = [item['chunk_paragraph'] for item in chunk_filtered_collection]

In [20]:
%%time
embedding_model.to('cuda')
text_chunk_embeddings = embedding_model.encode(chunk_paragraph_batch, batch_size = 10, convert_to_tensor = True)
text_chunk_embeddings

CPU times: total: 2min 53s
Wall time: 1min 30s


tensor([[ 0.0170,  0.0346, -0.0160,  ..., -0.0053,  0.0476, -0.0381],
        [ 0.0187,  0.0161, -0.0547,  ...,  0.0029,  0.0111, -0.0091],
        [-0.0161,  0.0439, -0.0165,  ..., -0.0552, -0.0096,  0.0129],
        ...,
        [ 0.0387,  0.0327, -0.0413,  ...,  0.0312,  0.0661, -0.0337],
        [ 0.0312,  0.0312, -0.0283,  ...,  0.0445, -0.0362, -0.0305],
        [ 0.0362,  0.0349, -0.0226,  ..., -0.0085,  0.0026, -0.0245]],
       device='cuda:0')

In [ ]:
#Save embeddings to a file
#chunk_filtered_collection[34]
embeddings_df = pd.DataFrame(chunk_filtered_collection)

In [ ]:
embeddings_df.to_csv('embeddings_df.csv')

## Second part is to do RAG Search and Answer

Goal - retrieve relevent data absed on query and use that data to augment input to LLM so it can generate the output based on that data\
Comparing embeddings - similarity search, vector search, semantic search

I have cloded the notebook so need to create embedding model again ignore

In [22]:
import torch
from sentence_transformers import util, SentenceTransformer
device = "cuda" if torch.cuda.is_available() else 'cpu'
embedding_model = SentenceTransformer(model_name_or_path = 'all-mpnet-base-v2', device = device)

#If reading from csv you will have to convert the embeddings to np.array/correct format and also need to make the dorrect datatype eg. torch.float32.
# I had to rerun the whole thing so fine but yaad rakh chutiye
text_chunk_embeddings.shape


torch.Size([1180, 768])

Embedding model is ready, now need to build a semantic search pipeline\
essentially query a text and get back passage/paragraph from textbook that is relevant.

steps:

1. give a query string
2. convert query string into embedding (using the same embedding model and device we used for the pdf data - all-mpnet-base-v2, cuda)
3. perform dot product or cosine similarity to get top k matches with ranks (we will call ranks based on similarity score we get. that should be a number the higher the better).

In [23]:
#This is a query
query = 'distributed file system'

#we need to embed the query
query_embedding = embedding_model.encode(query, convert_to_tensor= True).to("cuda")

#we are using dot product
from time import perf_counter as timer
strat_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=text_chunk_embeddings)[0]
end_time = timer()

print(f"[INFO] Time taken to get scores on {len(text_chunk_embeddings)} embeddings: {end_time - strat_time:.5f} seconds.")

#we want the top 10 results

top_results = torch.topk(dot_scores, k=10)
top_results

[INFO] Time taken to get scores on 1180 embeddings: 0.00168 seconds.


torch.return_types.topk(
values=tensor([0.6441, 0.6224, 0.6221, 0.6095, 0.5983, 0.5607, 0.5604, 0.5448, 0.5443,
        0.5302], device='cuda:0'),
indices=tensor([ 810,   20,  311,  809,    7,  398, 1168,  445,  313,  783],
       device='cuda:0'))

In [24]:
#We have our old list of dictionary where we can match these indices to check the actual paragraph chunks from the pdfs
chunk_filtered_collection[810]

#top_results


{'page_no': 398,
 'chunk_paragraph': 'Shared-disk storage is implemented by a centralized storage appliance, often using custom hardware and special network infrastructure such as Fibre Channel. On the other hand, the shared-nothing approach requires no special hardware, only computers connected by a conventional datacenter network. HDFS consists of a daemon process running on each machine, exposing a network service that allows other nodes to access files stored on that machine (assuming that every general-purpose machine in a datacenter has some disks attached to it). A central server called the NameNode keeps track of which file blocks are stored on which machine. Thus, HDFS conceptually creates one big filesystem that can use the space on the disks of all machines running the daemon. In order to tolerate machine and disk failures, file blocks are replicated on multiple machines. Replication may mean simply several copies of the same data on multiple machines, as in Chapter 5, or an

Searching over embeddings is fast but you will need to create indexes if you have huge embedding data\
you can use faiss library which helps in 'approximate nearest neighbor search' (ANN)

In [26]:
#Cleaning the output so that it is formatted well

import textwrap

def wrap_text(text, wrap_length=100):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

query = 'distributed file system'
print(f"Query:{query}\n")
print("results:")

for score, index in zip(top_results[0], top_results[1]):
    print(f"score:{score}")
    print("Text:")
    #print(chunk_filtered_collection[index]['chunk_paragraph'])
    wrap_text(chunk_filtered_collection[index]['chunk_paragraph'])
    

Query:distributed file system

results:
score:0.6440657377243042
Text:
Shared-disk storage is implemented by a centralized storage appliance, often using custom hardware
and special network infrastructure such as Fibre Channel. On the other hand, the shared-nothing
approach requires no special hardware, only computers connected by a conventional datacenter
network. HDFS consists of a daemon process running on each machine, exposing a network service that
allows other nodes to access files stored on that machine (assuming that every general-purpose
machine in a datacenter has some disks attached to it). A central server called the NameNode keeps
track of which file blocks are stored on which machine. Thus, HDFS conceptually creates one big
filesystem that can use the space on the disks of all machines running the daemon. In order to
tolerate machine and disk failures, file blocks are replicated on multiple machines. Replication may
mean simply several copies of the same data on multiple

The results can be potentially improved by using a reranking model.\
(a model that is trained to rank the results eg. the 10 that we got and rank them based on the usefullness)

can use mixedbreadai reranker)

In [30]:
#Making a function based on the code above

def retrieve_data(query: str, embeddings: torch.tensor, model: SentenceTransformer=embedding_model, k_resources_return: int=10, print_time: bool = True):

    query_embedding = embedding_model.encode(query, convert_to_tensor= True)
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - strat_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, k=k_resources_return)
    return scores, indices



retrieve_data(query='hive database for big data', embeddings = text_chunk_embeddings)




    

[INFO] Time taken to get scores on 1180 embeddings: 0.00168 seconds.


(tensor([0.6388, 0.5393, 0.5257, 0.5212, 0.5125, 0.5036, 0.5029, 0.4994, 0.4972,
         0.4943], device='cuda:0'),
 tensor([ 880,  235, 1162,  811,  844,  215,  884,  237,  852,  847],
        device='cuda:0'))

In [28]:
#Checking the local VRAM

gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes/(2**30))
print(f"Local GPU memory: {gpu_memory_gb}GB")

Local GPU memory: 8GB


In [1]:
!nvidia-smi

Sun May  4 14:37:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.41                 Driver Version: 566.41         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   69C    P8             23W /   35W |     701MiB /   8192MiB |     23%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Loading LLM locally:
1. Selecting an appropriate model based on your VRAM
2. Quantization config (what precision to load your model - 16bit, 8bit, 4bit etc)
3. Model ID - for telling the transformer what model/tokenizer to load
4. Tokenizer - turns text to numbers ready for LLM (different than embedding model)
5. LLM Model - for generation of the text we want

there are also different ways to speed up the model eg the token generation using "flash attention 2" which uses the attention module in transformers to speed things up (will need compute power of 8.0+ so can check on the nvidia website for GPU)

In [29]:
torch.cuda.get_device_capability(0) #0 is for which device I only have 1 device so 0

(8, 6)

We will need to download the LLM model using HuggingFace website. The model selection needs to be for HF transformer\
Also 

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'meta-llama/Llama-3.2-3B-Instruct'
access_token = ''  # Replace with your actual token
#get this by creating HF account and find tokens in your profile

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


C:\Users\yashb\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.30s/it]
C:\Users\yashb\anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yashb\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activat

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [2]:
local_path = "./llama-3-3b-instruct"

# Save locally
tokenizer.save_pretrained(local_path)
model.save_pretrained(local_path)


In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
local_path = "./llama-3-3b-instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.float16)
#need to pass this as a parameter in AutoModelForCausalLM while loading the model but for now we do not need to use this
#Also models come with bfloat16 dtype but bfloat is not working for my gpu so will need to convert it to float16
use_quantization_config = True

#Implementing flast attention 2 (which is now compatible with pytorch)

if is_flash_attn_2_available() and torch.cuda.get_device_capability(0)[0] >=8:
    attn_implementation = 'flash_attention_2'
else:
    attn_implementation = 'sdpa'

#Pick the model we want to use
#model_id = model_id  
model_id = local_path
#model_id = google/gemma-7b-it or whatever. I am using gemma-7b-quant which I downloaded already and is quantized so no need for quantization here.

#Instentiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = model_id)

#Instantiate the LLM Model
llm_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = model_id,
    torch_dtype = torch.float16,
    quantization_config = quantization_config if use_quantization_config else None,
    low_cpu_mem_usage = False,
    attn_implementation = attn_implementation
)

if not use_quantization_config:
    llm_model.to('cuda')

Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.10s/it]


In [1]:
!nvidia-smi

Sat May  3 21:35:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.41                 Driver Version: 566.41         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   67C    P8             23W /   35W |     321MiB /   8192MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
llm_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,)

In [9]:
print("Flash Attention 2 available:", is_flash_attn_2_available())
print("CUDA Capability:", torch.cuda.get_device_capability(0))
print("Using PyTorch version:", torch.__version__)

Flash Attention 2 available: False
CUDA Capability: (8, 6)
Using PyTorch version: 2.3.1+cu121


In [12]:
pip install flash-attn

     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ------------ --------------------------- 1.8/6.0 MB 11.2 MB/s eta 0:00:01
     ----------------------------- ---------- 4.5/6.0 MB 12.2 MB/s eta 0:00:01
     ---------------------------------------- 6.0/6.0 MB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\yashb\\AppData\\Local\\Temp\\pip-install-re2olaza\\flash-attn_af51ca497c7d4e569bd29601f4df8da0\\csrc\\composable_kernel\\library\\include\\ck\\library\\tensor_operation_instance\\gpu\\grouped_conv_bwd_weight\\device_grouped_conv_bwd_weight_two_stage_xdl_instance.hpp'



There is some issue with the flash attn so check this one later and still using the default

## Playing with the model

In [13]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

1803463680

In [16]:
# Set model to eval mode
llm_model.eval()

# Function to ask questions
def ask_question(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.95,
            temperature=0.7
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example
question = "What is the capital of France?"
response = ask_question(question)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is the capital of France? Paris
What is the capital of France?
Yes, that's correct! Paris is indeed the capital of France. Well done!

(Note: This question is often used as a way to test the user's knowledge of geography, particularly in relation to France. The question itself is simple, but it requires the user to recall and apply their knowledge of France's capital city.) 

**Explanation:** 
The question is asking for the capital city of France. The correct answer is Paris, which is a well-known fact in many parts of the world. The question is designed to be straightforward and easy to answer, making it a good choice for a multiple-choice question.

**Correct answer:** Paris

**Incorrect answers:** London, Berlin, Rome, etc.

**Explanation:** These cities are all capitals of other countries, but not France. The incorrect answers are designed to be plausible, but not accurate, making it easy for the user to make a mistake. 

This type of question is often used in assessments, qui

In [17]:
# Example
question = "Whats the distance between India and USA?"
response = ask_question(question)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Whats the distance between India and USA? 1000 miles?
No, not quite. The distance between India and USA is approximately 9,100 miles (14,500 km). To put it in perspective, it's roughly 14 times the distance between New York City and Los Angeles. India is a large country, and the United States is a vast continent. The distance between them is significant, making air travel a relatively long journey.

Distance between India and USA: 9,100 miles (14,500 km)

Note: The distance may vary depending on the specific locations within India and USA. This value is an approximate average distance between the two countries.


In [18]:
#This is the actual prompt formatting of Gemma, Llama and Mistral etc
#Still just playing around
def format_prompt(system_message, user_input, history=[]):
    prompt = "<|begin_of_text|>"

    if system_message:
        prompt += f"<|start_header_id|>system<|end_header_id|>\n{system_message}<|eot_id|>\n"

    for turn in history:
        prompt += f"<|start_header_id|>user<|end_header_id|>\n{turn['user']}<|eot_id|>\n"
        prompt += f"<|start_header_id|>assistant<|end_header_id|>\n{turn['assistant']}<|eot_id|>\n"

    prompt += f"<|start_header_id|>user<|end_header_id|>\n{user_input}<|eot_id|>\n"
    prompt += f"<|start_header_id|>assistant<|end_header_id|>\n"

    return prompt


In [19]:
system = "You are a helpful AI assistant."
history = []  # Append turns like {'user': '...', 'assistant': '...'}
user_input = "What's the tallest mountain in the world?"

prompt = format_prompt(system, user_input, history)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Parse out just the assistant's response
response = output_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].split("<|eot_id|>")[0].strip()
print(response)

# Update history
history.append({'user': user_input, 'assistant': response})


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


system
You are a helpful AI assistant.
user
What's the tallest mountain in the world?
assistant
The tallest mountain in the world is Mount Everest, located in the Himalayas on the border between Nepal and Tibet, China. Its height is approximately 8,848.86 meters (29,031.7 feet) above sea level.


In [20]:
history

[{'user': "What's the tallest mountain in the world?",
  'assistant': "system\nYou are a helpful AI assistant.\nuser\nWhat's the tallest mountain in the world?\nassistant\nThe tallest mountain in the world is Mount Everest, located in the Himalayas on the border between Nepal and Tibet, China. Its height is approximately 8,848.86 meters (29,031.7 feet) above sea level."}]

Cleaning the space in GPU 

In [32]:
#cleaning GPU space

del llm_model
del tokenizer
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
import gc
gc.collect()

26148

In [6]:
#Lets create a simple template for prompt 

input_text = 'How are the distributed databases useful for Big data?'
print(f"user: {input_text}\n")

#dialogue template for instruction tuned model

dialogue = [
    {
        "role":"user",
        "content":input_text
    }
]
prompt = tokenizer.apply_chat_template(conversation = dialogue, tokenize = False, add_generation_prompt = True)
#print(f"prompt(formatted): {prompt}\n")
input_id = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = llm_model.generate(
        **input_id,
        max_new_tokens=200
    )

output_text = tokenizer.decode(outputs[0])
print(f"llm: {output_text}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


user: How are the distributed databases useful for Big data?



C:\Users\yashb\anaconda3\envs\rag\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


llm: <|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

How are the distributed databases useful for Big data?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Distributed databases are particularly useful for handling Big Data because they can scale horizontally, allowing them to handle large volumes of data that can be too large for a single database to manage. Here are some reasons why distributed databases are useful for Big Data:

1. **Scalability**: Distributed databases can scale horizontally, adding more nodes to the cluster as needed to handle increasing amounts of data. This allows them to handle large volumes of data that can't be handled by a single database.
2. **High performance**: Distributed databases can distribute the load across multiple nodes, improving performance and reducing the time it takes to process and analyze l

In [24]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./llama-3-3b-instruct', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_special_token_2|>", rstrip=False

## Augmenting prompt with context items

Prompting techniques:
1. give clear instructions
2. give few examples of i/p / o/p
3. give room to think\
using the function we created earlier where we put the query and get the value and index, then using that index to get the human 
readeable values from the list of dict we created 

In [46]:
def prompt_formatter(query:str, context_items: list[dict]) -> str:
    context = '- ' + '\n- '.join(item['chunk_paragraph'] for item in context_items)
    base_prompt = """ Based on the following context items, please answer the query.
    context_items:
    {context}
    query:{query}
    answer:
    """
    prompt = base_prompt.format(context = context, query = query)
    
    return prompt

query = 'How does distributed databases help in big data processing?'
query = 'what is a way to connect Kafka Pipeline to data processing application?'

scores, index = retrieve_data(query=query, k_resources_return =5, embeddings= text_chunk_embeddings, print_time = False)
context_items = [chunk_filtered_collection[i] for i in index]

prompt = prompt_formatter(query, context_items)
print(f"query: {query}")
print(f"prompt: {prompt}")




query: what is a way to connect Kafka Pipeline to data processing application?
prompt:  Based on the following context items, please answer the query.
    context_items:
    - doi: 10.1145/568522.568525 [100] Adam Warski: “Kafka Streams – How Does It Fit the Stream Processing Landscape?,” softwaremill.com, June 1, 2016. Summary | 487
- model as a table into which transactions can insert tuples, but which cannot be queried. The stream then consists of the log of tuples that committed transactions have written to this special table, in the order they were committed. External consumers can asynchronously consume this log and use it to update derived data systems. Kafka Connect [41] is an effort to integrate change data capture tools for a wide range of database systems with Kafka. Once the stream of change events is in Kafka, it can be used to update derived data systems such as search indexes, and also feed into stream processing systems as discussed later in this chapter. Event Sourcing

In [47]:
%%time

input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')

outputs = llm_model.generate(**input_ids, temperature=0.7, do_sample=True, max_new_tokens=250)
outputs_text= tokenizer.decode(outputs[0])
print(f"query: {query}")
print(f"rag_answer:\m{outputs_text.replace(prompt, '')}")



<unknown>:6: SyntaxWarning: invalid escape sequence '\m'
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


query: what is a way to connect Kafka Pipeline to data processing application?
rag_answer:\m<|begin_of_text|> There are several ways to connect a Kafka pipeline to a data processing application. Here are a few options:

     1.  **Change Data Capture (CDC)**: Kafka Connect can be used to integrate change data capture tools with Kafka. This allows the stream of change events to be used to update derived data systems such as search indexes and feed into stream processing systems.

     2.  **Event Sourcing**: Event sourcing involves storing all changes to the application state as a log of change events. This can be used to implement a data processing pipeline that can be used to update derived data systems.

     3.  **Apache Flink**: Apache Flink is a stream processing framework that can be used to process data in real-time. It can be used to implement a data processing pipeline that can be connected to a Kafka pipeline.

     4.  **Apache Samza**: Apache Samza is a stream processing fr

In [53]:
%%time
def prompt_formatter2(query:str, context_items: list[dict]) -> str:
    context = '- ' + '\n- '.join(item['chunk_paragraph'] for item in context_items)
    base_prompt = """ Based on the following context items, please answer the query.
    give yourself room to think by extracting relevant passages from the context before answering the query.
    Dont return the thinking only return the answer.
    Make sure your answers are as explanatory as possible.
    Use the following examples as the ideal answer style
    \nExample 1:
    Query: What drove the adoption of NoSQL Databases?
    Answer: A need for greater scalability than relational databases can easily achieve, including very large datasets or very high write throughput, a widespread preference for free and open source software over commercial database products, specialized query operations that are not well supported by the relational model and frustration with the restrictiveness of relational schemas, and a desire for a more dynamic and expressive data model are some of the driving forces that let to adaptation of NoSQL Databases.
    \nExample 2:
    Query: Can you give some info on Message Brokers?
    Answer: Message Brokers are widely used alternative is to send messages (also known as a  message queue), which is essentially a kind of database that is optimized for handling message streams. It runs as a server, with producers and consumers connecting  to it as clients. Producers write messages to the broker, and consumers receive them by reading them from the broker.
    Now use the following context item to answer the query
    context_items:
    {context}
    query:{query}
    answer:
    """
    base_prompt = base_prompt.format(context = context, query = query)
    #dialogue template for instruction tuned model

    dialogue = [
        {
            "role":"user",
            "content":base_prompt
        }
    ]
    #Apply chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue, tokenize=False, add_generation_prompt=True) #will tokenize it later like last time
    return prompt

query = 'Can you give some info on Graph Databases?'

scores, index = retrieve_data(query=query, k_resources_return =5, embeddings= text_chunk_embeddings, print_time = False)
context_items = [chunk_filtered_collection[i] for i in index]

prompt = prompt_formatter2(query, context_items)
#print(prompt)


input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')

outputs = llm_model.generate(**input_ids, temperature=0.7, do_sample=True, max_new_tokens=250)
outputs_text= tokenizer.decode(outputs[0])
print(f"query: {query}")
print(f"rag_answer:\m{outputs_text.replace(prompt, '')}")

<unknown>:47: SyntaxWarning: invalid escape sequence '\m'
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


query: Can you give some info on Graph Databases?
rag_answer:\m<|begin_of_text|>Graph Databases are a type of NoSQL database that is optimized for handling complex data relationships and queries. They consist of two main types of objects: vertices (also known as nodes or entities) and edges (also known as relationships or arcs). Each vertex represents a unique entity, such as a person, web page, or junction, while edges represent the relationships between these entities. Graph databases provide a flexible and dynamic way to store and query data, making them suitable for applications with complex relationships, such as social networks, web graphs, and road or rail networks. They support various query languages, including Cypher, SPARQL, and Datalog, and are particularly useful for evolvability, allowing easy extension of the database as the application's data structure evolves. Graph databases are particularly useful in applications where there are many-to-many relationships, and can be

Still need to understand more about the dialogue for chat style\
\
Creating the function to play around in chat like structure